In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Activation, add, Flatten, AveragePooling2D, concatenate
from keras.models import Model

from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from keras.utils import np_utils

width = 32
height = 32

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

https://www.tensorflow.org/api_docs/python/tf/keras/applications/InceptionV3

1. label 을 one-hot vector 로 변환

In [ ]:
num_classes = 10
y_train = np_utlls.to_categorical(y_train)
y_test = np_utlls.to_categorical(y_test)


2. list slicing 으로  train image / validation image 분리 

In [ ]:
valldation_images, valldation_labels = x_trin[:500], y_trin[:500]
train_images, train_labels = x_train[500:], y_train[500:]

In [ ]:
train_images.shape

(49500, 32, 32, 3)

3. Inception module 정의

In [ ]:
def inception(x, filters): 
    pre_layer = x
    
    f1,f2,f3,f4 = filters

    # 1x1
    
    # 1x1 & 3x3

    # 1x1 & 5x5
    
    # pooling & 1x1

    # output = [None(batch_size), w,h,c], c 기준 concatenate (axis = -1 )
    concat = concatenate([conv1, conv2, conv3, max_pool], axis=-1)
    
    return concat
    

4. GoogleNet (Incepceion V1) 모델 정의

In [ ]:
input_shape =  x_train[0].shape
inputs = Input(shape= input_shape )

# conv랑 batch 사이에 max pooling 들어가야 하나, cifar 데이터에선 크기 너무 줄어들어서 뺐음
x = Conv2D(64, kernel_size=(7,7), strides=2, padding='same', activation='relu')(inputs)
x = BatchNormalization()(x)
x = Conv2D(192, kernel_size=(3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x) 

#inception (input, [#1x1conv, #3x3conv, #5x5conv, #1x1reduction] )

# inception 3a
x= inception(x, [64 ,128 ,32 , 32 ])
# inception 3b
x= inception(x, [128 ,192 ,96 ,64 ])
x = MaxPooling2D(pool_size=(3,3),strides = 2, padding='same')(x)


# inception 4a
x= inception(x, [192 , 208 , 48 ,64 ])

#aux1
aux1 = AveragePooling2D(pool_size=(5,5), strides = 3, padding= 'valid')(x)
aux1 = Conv2D(128, kernel_size = (1,1), padding='same', activation = 'relu')(aux1)
aux1 = Flatten()(aux1)
aux1 = Dense(512, activation ='relu')(aux1)
aux1 = Dense(10, activation = 'softmax')(aux1)

# inception 4b
x= inception(x, [160 , 224  ,64  ,64 ])
# inception 4c
x= inception(x, [128 ,256  ,64  ,64 ])
# inception 4d
x= inception(x, [112 ,288  ,64  ,64 ])

#aux2
aux2 = AveragePooling2D(pool_size=(5,5), strides = 3, padding= 'valid')(x)
aux2 = Conv2D(128, kernel_size = (1,1), padding='same', activation = 'relu')(aux2)
aux2 = Flatten()(aux2)
aux2 = Dense(832, activation ='relu')(aux2)
aux2 = Dense(10, activation = 'softmax')(aux2)


# inception 4e
x= inception(x, [256 ,320  ,128  ,128 ])
x = MaxPooling2D(pool_size=(3,3),strides = 2, padding='same')(x)


# inception 5a
x= inception(x, [256 ,320  ,128  ,128 ])
# inception 5b
x= inception(x, [384 ,384  ,128  ,128 ])

x = AveragePooling2D(pool_size = (4,4), padding='valid')(x)
x = Dropout(0.4)(x)

outputs = Dense( 10 , activation='softmax')(x)



In [ ]:
model = Model( inputs = inputs, outputs=[aux1, aux2, outputs] )


# adam optimizer, categorical crossentropy, loss weight ratio (0.3,0.3,1.0), metric: accuracy
model.compile( optimizer = 'adam', loss='categorical_crossentropy', loss_weights=[0.3,0.3,1.0], metrics=['accuracy'] ) 


In [ ]:
model.summary()

4. ImageDagaGenerator 로 normalization 및 augmentation 적용 (shear , zoom , horizontal flip 적용 )

In [ ]:
train_datagen = imageDataGeneratop(rescale = 1./255, shear_range = 0.2, zoom_range =)

validation_datagen =

5. batch 32, epoch 10 으로 학습

In [ ]:
history = model.fit()

6. train data, validation data 각각에 대해 마지막 layer의 loss 와 accuracy 

---



In [ ]:
import matplotlib.pyplot as plt

acc = 
val_acc = 

loss =
val_loss = 

epochs_range = range(10)


# accuracy
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(  label='Training Accuracy')
plt.plot(  label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# loss
plt.subplot(1, 2, 2)
plt.plot(  label='Training Loss')
plt.plot(   label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()